# Little Washita ParFlow-CLM Annotated Example
This workbook illustrates how to setup and run a watershed simulation. The test problem we are running is of the [Little Washita watershed](https://www.ars.usda.gov/plains-area/el-reno-ok/grazinglands-research-laboratory/agroclimate-and-natural-resources-research/docs/little-washita-river-experimental-watershed/) which has been a USDA research watershed for many decades and the site of several field campaigns and modeling studies. 

We will be setting up the domain using a terrain following grid and a solid file.  The subsurface extends more than 1000m deep using layers of increasing thickness ranging from 0.1 m to 1000 m created using the variable dz option. 

This simulation is running with the land surface model CLM and is set to run for 72 hours using meteorological forcing files provided in this repo. 

Note that the simulation starts from an initial pressure file. This pressure file is derived using a spin up simulation that determines the steady state groundwater configuration that this run will start from. 

**Simulation Inputs**
1. Solid file: used to define the shape of the domain 
2. Indicator file: used to define where the different geologic units are in the grid
3. Slope files: slope x and slope y files define the topographic slopes of each grid cell in the x and y directions
4. Initial pressure:  a pressure file created from a spinup run of the model (refer to other Little Washita exercises for spinup)

### 1. Import required libraries and functions 
Before you can run this workbook you will need to make sure that you have parflow and pftools installed. PFtools is the python package that you need in order to build a ParFlow run through python and ParFlow is the actual simulation code. 

Refer to the main readme of this repo for instructions on getting your modeling environment setup.

In [1]:
import os
import numpy as np
from parflow import Run 
import shutil
from parflow.tools.fs import mkdir, cp, get_absolute_path, exists
from parflow.tools.settings import set_working_directory

## 2. Setup your run directory and initialize your model object

In [2]:
# Name your ParFLow run -- note that all of your output files will have this prefix
runname = 'LW_CLM'

# Create a directory in the outputs folder for this run
run_dir = get_absolute_path(f'outputs/{runname}')
mkdir(run_dir)
print(run_dir)

# create your Parflow model object. For starters we are just goin to set the file version and the run directory we'll add more later
# note that the model will run from the run_dir so all input files should be in the run dir or paths should be specified relative to run_dir
model = Run(runname, run_dir)
model.FileVersion = 4

#copy the model inputs for the simulation into the run directory
#NOTE: you dont have to copy everything into the run directory if you don't want, you can also point to input files in other directories in a simulation if you prefer
input_dir= os.path.join(os.getcwd(), 'model_inputs')
files=os.listdir(input_dir)
for fname in files:
    shutil.copy(os.path.join(input_dir,fname), run_dir)

/data/outputs/LW_CLM


## 3. Setup the computational grid
Notes:
1. In this step and the subsequent steps we are doing all of this by adding ParFlow keys to the `model` object that we created above.  
2. When you run this cell nothing is actually runing in your model, we are just building up a database of keys that define the model we will be running. 

### 3.1 processor toppology and computational grid

In [3]:
# Processor topology: This is the way that the problem will be split across processors if you want to run in parallel
# The domain is divided in x,y and z dimensions by P, Q and R. The total number of processors is P*Q*R.
model.Process.Topology.P = 1
model.Process.Topology.Q = 1
model.Process.Topology.R = 1

#Locate the origin in the domain.
model.ComputationalGrid.Lower.X = 0.0
model.ComputationalGrid.Lower.Y = 0.0
model.ComputationalGrid.Lower.Z = 0.0

# Define the size of each grid cell. The length units are the same as those on hydraulic conductivity, here that is meters.
model.ComputationalGrid.DX = 1000.0
model.ComputationalGrid.DY = 1000.0
model.ComputationalGrid.DZ = 200.0

# Define the number of grid blocks in the domain.
model.ComputationalGrid.NX = 64
model.ComputationalGrid.NY = 32
model.ComputationalGrid.NZ = 10

### 3.2 Geometries
In ParFlow we use `geometries` to define our problem space. 

First, we declear the names of the `geometries` we plan to define. In this case we have two: `solid_input` (used to define our watershed boundary) and `indi_input` (used to define the subsurface units). 

Next, for each of the declared `geometries` we specify a type and setup names within the geometry. 

**Solid File Geometry:** Our first geomety is a solid file.

In [4]:
#Declare the geometries that you will use for the problem
model.GeomInput.Names = "solid_input indi_input"

#Define the solid_input geometry.  
#Note the naming convention here GeomInput.{GeomName}.key
model.GeomInput.solid_input.InputType = "SolidFile"
model.GeomInput.solid_input.GeomNames = "domain"
model.GeomInput.solid_input.FileName = "LW.pfsol"

#First set the name for your `Domain` and setup the patches for this domain
model.Domain.GeomName = "domain"
model.Geom.domain.Patches = "top bottom side"

**Indicator Geometry**
Now we setup the indicator file. The indicator file is a gridded input that has an integer value for every grid cell in the domain designating what geologic unit it belongs to. 

The GeomNames list should include a name for every integer value in your indicator file. In this example we have thirteen soil units (s1-s13) and ten geologic units (g1-g13). The FileName points to the indicator file that ParFlow will read. Recall that this file is copied into the run directory at the start of the script.

For every name in the GeomNames list we define the corresponding value in the indicator file. 
or example, here we are saying that our first soil unit (s1) is represented by the number “1" in the indicator file, while the first geologic unit (g1) is represented by the number “19". Note: that the integers used in the indicator file do not need to be consecutive. 

In [5]:
model.GeomInput.indi_input.InputType =   "IndicatorField"
model.GeomInput.indi_input.GeomNames = "s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13 g1 g2 g3 g4 g5 g6 g7 g8 g9 g10"
model.Geom.indi_input.FileName = "Indicator_LW_USGS_Bedrock.pfb"

model.GeomInput.s1.Value =    1
model.GeomInput.s2.Value =    2
model.GeomInput.s3.Value =    3
model.GeomInput.s4.Value =    4
model.GeomInput.s5.Value =    5
model.GeomInput.s6.Value =    6
model.GeomInput.s7.Value =    7
model.GeomInput.s8.Value =    8
model.GeomInput.s9.Value =    9
model.GeomInput.s10.Value =   10
model.GeomInput.s11.Value =   11
model.GeomInput.s12.Value =   12
model.GeomInput.s13.Value =   13

model.GeomInput.g1.Value =    19
model.GeomInput.g2.Value =    20
model.GeomInput.g3.Value =    21
model.GeomInput.g4.Value =    22
model.GeomInput.g5.Value =    23
model.GeomInput.g6.Value =    24
model.GeomInput.g7.Value =    25
model.GeomInput.g8.Value =    26
model.GeomInput.g9.Value =    27
model.GeomInput.g10.Value =    28

### 3.3 Variable ${\Delta z}$ 
Variable $\Delta z$ allows you to change the thickness of each of your z-layers, the default is a uniform thickness.  In this example, the `DZ` is 200.0 and the `NZ` is 10, meaning the 10 layers, will each be 200m thick by default. 

The `dzscale.Value` lines are numbers by which the uniform layer thicknesses, 200m in our case, can be scaled in the simulation. Layer 0 refers to the bottom layer which means that the bottom layer will have a thickness of 1000m (200 * 5)  and the The top layer (9) with a scalar of .0005 will have a thickness of 0.1m (.0005 * 200m)

In [6]:
model.Solver.Nonlinear.VariableDz = True
model.dzScale.GeomNames = "domain"
model.dzScale.Type = "nzList"
model.dzScale.nzListNumber = 10

model.Cell._0.dzScale.Value = 5
model.Cell._1.dzScale.Value = 0.5
model.Cell._2.dzScale.Value = 0.25
model.Cell._3.dzScale.Value = 0.125
model.Cell._4.dzScale.Value = 0.05
model.Cell._5.dzScale.Value = 0.025
model.Cell._6.dzScale.Value = 0.005
model.Cell._7.dzScale.Value = 0.003
model.Cell._8.dzScale.Value = 0.0015
model.Cell._9.dzScale.Value = 0.0005

### 3.4 Topographic slopes 
Next we define topographic slopes and values. These slope values were derived from a digital elevation model of the domain following the workflow in the PriorityFlow github package.  In this example we read the slope files in from .pfb files that were copied into the run directory at the start of this script.

In [7]:
model.TopoSlopesX.Type = "PFBFile"
model.TopoSlopesX.GeomNames = "domain"
model.TopoSlopesX.FileName = "slopex_LW.pfb"

model.TopoSlopesY.Type = "PFBFile"
model.TopoSlopesY.GeomNames = "domain"
model.TopoSlopesY.FileName = "slopey_LW.pfb"

## 4. Setup the surface and subsurface properties 
Next we will assign properties to the subsurface. Note that if a value is assigned more than once only the last assignment will be used.

Also, note that you do not need values for all of the geometries names defined above, as long a value is assigned to every active cell in the domain.  However, every geometry listed in, *e.g.* `Geom.Porosity.GeomNames` must have values assigned.  If you assign a domain value then any grid cell not included in another geometry will take the domain value. 

### 4.1 Permeability
Here we add permeability ($k$) data to the domain using the geometries from the indicator file. As [described below](##5.-phases-contaminants,-gravity-and-wells), we convert to *Hydraulic Conductivity*, $K$ in $[LT^-1]$ units for our simulation. Note that there are no units in the script but the units you use for $K$ define the length and time units for the rest of the problem.  For this problem we are expressing values as $[m/hr]$

In [8]:
model.Geom.Perm.Names = "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13 g1 g2 g3 g4 g5 g6 g7 g8 g9 g10"

model.Geom.domain.Perm.Type = "Constant"
model.Geom.domain.Perm.Value = 0.02

model.Geom.s1.Perm.Type = "Constant"
model.Geom.s1.Perm.Value = 0.269022595

model.Geom.s2.Perm.Type = "Constant"
model.Geom.s2.Perm.Value = 0.043630356

model.Geom.s3.Perm.Type = "Constant"
model.Geom.s3.Perm.Value = 0.015841225

model.Geom.s4.Perm.Type = "Constant"
model.Geom.s4.Perm.Value = 0.007582087

model.Geom.s5.Perm.Type = "Constant"
model.Geom.s5.Perm.Value = 0.01818816

model.Geom.s6.Perm.Type = "Constant"
model.Geom.s6.Perm.Value = 0.005009435

model.Geom.s7.Perm.Type = "Constant"
model.Geom.s7.Perm.Value = 0.005492736

model.Geom.s8.Perm.Type = "Constant"
model.Geom.s8.Perm.Value = 0.004675077

model.Geom.s9.Perm.Type = "Constant"
model.Geom.s9.Perm.Value = 0.003386794

model.Geom.s10.Perm.Type = "Constant"
model.Geom.s10.Perm.Value = 0.004783973

model.Geom.s11.Perm.Type = "Constant"
model.Geom.s11.Perm.Value = 0.003979136

model.Geom.s12.Perm.Type = "Constant"
model.Geom.s12.Perm.Value = 0.006162952

model.Geom.s13.Perm.Type = "Constant"
model.Geom.s13.Perm.Value = 0.005009435

model.Geom.g1.Perm.Type = "Constant"
model.Geom.g1.Perm.Value = 5e-3

model.Geom.g2.Perm.Type = "Constant"
model.Geom.g2.Perm.Value = 1e-2

model.Geom.g3.Perm.Type = "Constant"
model.Geom.g3.Perm.Value = 2e-2

model.Geom.g4.Perm.Type = "Constant"
model.Geom.g4.Perm.Value = 3e-2

model.Geom.g5.Perm.Type = "Constant"
model.Geom.g5.Perm.Value = 4e-2

model.Geom.g6.Perm.Type = "Constant"
model.Geom.g6.Perm.Value = 5e-2

model.Geom.g7.Perm.Type = "Constant"
model.Geom.g7.Perm.Value = 6e-2

model.Geom.g8.Perm.Type = "Constant"
model.Geom.g8.Perm.Value = 8e-2

model.Geom.g9.Perm.Type = "Constant"
model.Geom.g9.Perm.Value = 0.1

model.Geom.g10.Perm.Type = "Constant"
model.Geom.g10.Perm.Value = 0.2

### 4.2 Permeability tensor
The following section allows you to specify the permeability tensor. In the case below, permeability is symmetric in all directions ($x$, $y$, and $z$) and therefore each is set to 1.0. Also note that we just specify this once for the whole domain because we want isotropic permeability everywhere. You can specify different tensors for different units by repeating these lines with different Geom.Names.

In [9]:
model.Perm.TensorType = "TensorByGeom"
model.Geom.Perm.TensorByGeom.Names = "domain"
model.Geom.domain.Perm.TensorValX = 1.0
model.Geom.domain.Perm.TensorValY = 1.0
model.Geom.domain.Perm.TensorValZ = 1.0

### 4.3  Specific Storage
Next we set the specific storage. Here again we specify one value for the whole domain but these lines can be easily repeated to set different values for different units.

In [10]:
model.SpecificStorage.Type = "Constant"
model.SpecificStorage.GeomNames = "domain"
model.Geom.domain.SpecificStorage.Value = 0.0001

### 4.4 Porosity
Next, we assign the [porosity](https://parflow-docs.readthedocs.io/en/latest/keys.html#porosity). As with the permeability we assign different values for different indicator geometries. Here we assign values for all of our soil units but not for the geologic units, they will default to the domain value of 0.33.

In [11]:
model.Geom.Porosity.GeomNames = "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13"

model.Geom.domain.Porosity.Type = "Constant"
model.Geom.domain.Porosity.Value = 0.33

model.Geom.s1.Porosity.Type = "Constant"
model.Geom.s1.Porosity.Value = 0.375

model.Geom.s2.Porosity.Type = "Constant"
model.Geom.s2.Porosity.Value = 0.39

model.Geom.s3.Porosity.Type = "Constant"
model.Geom.s3.Porosity.Value = 0.387

model.Geom.s4.Porosity.Type = "Constant"
model.Geom.s4.Porosity.Value = 0.439

model.Geom.s5.Porosity.Type = "Constant"
model.Geom.s5.Porosity.Value = 0.489

model.Geom.s6.Porosity.Type = "Constant"
model.Geom.s6.Porosity.Value = 0.399

model.Geom.s7.Porosity.Type = "Constant"
model.Geom.s7.Porosity.Value = 0.384

model.Geom.s8.Porosity.Type = "Constant"
model.Geom.s8.Porosity.Value = 0.482

model.Geom.s9.Porosity.Type = "Constant"
model.Geom.s9.Porosity.Value = 0.442

model.Geom.s10.Porosity.Type = "Constant"
model.Geom.s10.Porosity.Value = 0.385

model.Geom.s11.Porosity.Type = "Constant"
model.Geom.s11.Porosity.Value = 0.481

model.Geom.s12.Porosity.Type = "Constant"
model.Geom.s12.Porosity.Value = 0.459

model.Geom.s13.Porosity.Type = "Constant"
model.Geom.s13.Porosity.Value = 0.399

### 4.5 Relative Permeability
Following the same approach as we did for Porosity we define the relative permeability inputs that will be used for Richards’ equation implementation (§ 6.1.20). Here we use VanGenuchten parameters. By using the `domain` geometry, we can set a background value of $\alpha$ and $n$ to one.  In this way any units not defined in the `Phase.RelPerm.GeomNames` will get the background domain value of 1.0 and 3.0

In [12]:
model.Phase.RelPerm.Type =              "VanGenuchten"
model.Phase.RelPerm.GeomNames =     "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13"

model.Geom.domain.RelPerm.Alpha =    1.0
model.Geom.domain.RelPerm.N =        3.0

model.Geom.s1.RelPerm.Alpha =        3.548
model.Geom.s1.RelPerm.N =            4.162

model.Geom.s2.RelPerm.Alpha =        3.467
model.Geom.s2.RelPerm.N =            2.738

model.Geom.s3.RelPerm.Alpha =        2.692
model.Geom.s3.RelPerm.N =            2.445

model.Geom.s4.RelPerm.Alpha =        0.501
model.Geom.s4.RelPerm.N =            2.659

model.Geom.s5.RelPerm.Alpha =        0.661
model.Geom.s5.RelPerm.N =            2.659

model.Geom.s6.RelPerm.Alpha =        1.122
model.Geom.s6.RelPerm.N =            2.479

model.Geom.s7.RelPerm.Alpha =        2.089
model.Geom.s7.RelPerm.N =            2.318

model.Geom.s8.RelPerm.Alpha =        0.832
model.Geom.s8.RelPerm.N =            2.514

model.Geom.s9.RelPerm.Alpha =        1.585
model.Geom.s9.RelPerm.N =            2.413

model.Geom.s10.RelPerm.Alpha =        3.311
model.Geom.s10.RelPerm.N =            2.202

model.Geom.s11.RelPerm.Alpha =        1.622
model.Geom.s11.RelPerm.N =            2.318

model.Geom.s12.RelPerm.Alpha =        1.514
model.Geom.s12.RelPerm.N =            2.259

model.Geom.s13.RelPerm.Alpha =        1.122
model.Geom.s13.RelPerm.N =            2.479

### 4.6 Saturation
Next we do the same thing for saturation (§ 6.1.23) again using the van Genuchten parameters. Unnamed units will get the background domain values assigned

In [13]:
model.Phase.Saturation.Type =             "VanGenuchten"
model.Phase.Saturation.GeomNames =         "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13"

model.Geom.domain.Saturation.Alpha =        1.0
model.Geom.domain.Saturation.N =            3.0
model.Geom.domain.Saturation.SRes =         0.001
model.Geom.domain.Saturation.SSat =         1.0

model.Geom.s1.Saturation.Alpha =        3.548
model.Geom.s1.Saturation.N =            4.162
model.Geom.s1.Saturation.SRes =         0.0001
model.Geom.s1.Saturation.SSat =         1.0

model.Geom.s2.Saturation.Alpha =        3.467
model.Geom.s2.Saturation.N =            2.738
model.Geom.s2.Saturation.SRes =         0.0001
model.Geom.s2.Saturation.SSat =         1.0

model.Geom.s3.Saturation.Alpha =        2.692
model.Geom.s3.Saturation.N =            2.445
model.Geom.s3.Saturation.SRes =         0.0001
model.Geom.s3.Saturation.SSat =         1.0

model.Geom.s4.Saturation.Alpha =        0.501
model.Geom.s4.Saturation.N =            2.659
model.Geom.s4.Saturation.SRes =         0.1
model.Geom.s4.Saturation.SSat =         1.0

model.Geom.s5.Saturation.Alpha =        0.661
model.Geom.s5.Saturation.N =            2.659
model.Geom.s5.Saturation.SRes =         0.0001
model.Geom.s5.Saturation.SSat =         1.0

model.Geom.s6.Saturation.Alpha =        1.122
model.Geom.s6.Saturation.N =            2.479
model.Geom.s6.Saturation.SRes =         0.0001
model.Geom.s6.Saturation.SSat =         1.0

model.Geom.s7.Saturation.Alpha =        2.089
model.Geom.s7.Saturation.N =            2.318
model.Geom.s7.Saturation.SRes =         0.0001
model.Geom.s7.Saturation.SSat =         1.0

model.Geom.s8.Saturation.Alpha =        0.832
model.Geom.s8.Saturation.N =            2.514
model.Geom.s8.Saturation.SRes =         0.0001
model.Geom.s8.Saturation.SSat =         1.0

model.Geom.s9.Saturation.Alpha =        1.585
model.Geom.s9.Saturation.N =            2.413
model.Geom.s9.Saturation.SRes =         0.0001
model.Geom.s9.Saturation.SSat =         1.0

model.Geom.s10.Saturation.Alpha =        3.311
model.Geom.s10.Saturation.N =            2.202
model.Geom.s10.Saturation.SRes =         0.0001
model.Geom.s10.Saturation.SSat =         1.0

model.Geom.s11.Saturation.Alpha =        1.622
model.Geom.s11.Saturation.N =            2.318
model.Geom.s11.Saturation.SRes =         0.0001
model.Geom.s11.Saturation.SSat =         1.0

model.Geom.s12.Saturation.Alpha =        1.514
model.Geom.s12.Saturation.N =            2.259
model.Geom.s12.Saturation.SRes =         0.0001
model.Geom.s12.Saturation.SSat =         1.0

model.Geom.s13.Saturation.Alpha =        1.122
model.Geom.s13.Saturation.N =            2.479
model.Geom.s13.Saturation.SRes =         0.0001
model.Geom.s13.Saturation.SSat =         1.0

### 4.7 Manning's coefficient
And now we define the Manning's Roughness Coefficient, $n$, as just one value for the whole domain in this example. This is a simplification for our example, the $n$ values can be assigned uniquely for every domain cell.

In [14]:
model.Mannings.Type = "Constant"
model.Mannings.GeomNames = "domain"
model.Mannings.Geom.domain.Value = 0.0000044

## 5. Phases contaminants, gravity and wells
**Phases:** ParFlow has the capability to deal with a multiphase system (using `Solver=IMPES`) and the [equations of flow](https://parflow-docs.readthedocs.io/en/latest/models.html#richards-equation) (in our case Richards' equation) are written using *intrinsic permeability*, $k$.  As shown below, intrinsic permeabilty is converted to *Hydraulic Conductivity* when we are solving for only one phase (water) such as in this example, but also very commonly in watershed applications.

$$
   K(p) =  \frac{{\bar k}k_r(p)\rho(p)}{\mu}
$$

As we stated earlier, we set density, $\rho(p)$, and viscosity, $\mu$,(and later gravity) equal to 1.0.  Again, this is merely a trick to solve for hydraulic conductivity $[LT^-1]$ and pressure, $p\,[L]$. If you were to set density and viscosity to their true values, the code would calculate everything in terms of $k$ (intrinsic permeability) which is in units of $[L^2]$. By using the normalized values instead, you effectively embed the conversion of $k$ to $K$ (hydraulic conductivity) within the input script. In this way, we get hydraulic conductivity, which is what we want for this problem.

**Contaminants:** This example does not include the ParFlow grid based transport scheme. Therefore we leave contaminants blank.

**Gravity:** As with density and viscosity, gravity is normalized here now to allow us to calculate in units of *pressure-head*, $\psi$, in length units, $[L]$ typically *m*, instead of pressure, $p$, in units of force per unit area typically *Pa*. This conversion is something like:
$$
   \psi =  \frac{p}{\rho(p)g}
$$
If we used the true value (in the $[L]$ and $[T]$ units of hydraulic conductivity) the code would be calculating permeability ($k$). Instead, we normalize so that the code calculates hydraulic conductivity ($K$).  Of course by making density constant (and one) here and above, we are solving for flow at a constant temperature.

**Wells:** Again, ParFlow has more capabilities than we are using here in this example. Note that since there are no well names listed here, ParFlow assumes we have no wells. If we had pumping wells, we would have to include them here, because they would affect the head distribution throughout our domain. See § 3.6.1 for an example of how to include pumping wells in this script.

**Phase sources:** Phase sources allows you to add sources other than wells and boundaries, but we do not have any so this key is constant, 0.0 over entire domain.

In [15]:
# Phases
model.Phase.Names = "water"
model.Phase.water.Density.Type = "Constant"
model.Phase.water.Density.Value = 1.0
model.Phase.water.Viscosity.Type = "Constant"
model.Phase.water.Viscosity.Value = 1.0
model.Phase.water.Mobility.Type = "Constant"
model.Phase.water.Mobility.Value = 1.0

# Contaminants
model.Contaminants.Names = ""

# Gravity
model.Gravity = 1.0

#Wells
model.Wells.Names = ""

# Phase Sources
model.PhaseSources.water.Type = "Constant"
model.PhaseSources.water.GeomNames = "domain"
model.PhaseSources.water.Geom.domain.Value = 0.0

## 6. Timing
The units of time are set by the *hydraulic conductivity*, $K$ units $[LT-1]$, in the case our units are *hours*. 


**Base Unit:** This specifies the base unit of time for all *Time Cycle* values (see below).  This is like a conversion between the *time step* and the cycle intervals. Time cycles intervals are expressed as multiples of this value. To keep things simple here we set it to 1.0. Because we expressed our hydraulic conductivity in units of $[m/hr]$ in this example this means that our base unit of time is 1 *h*.

**Start Count:** The time step number that your first output (the initial condition) will use.  Because we are starting from scratch we set this to 0. If we were restarting a run we would set this to the last time step of your previous simulation.

**Start and Stop times:**  `StartTime` and `StopTime` specify the start and stop times for the simulation in units of *h*. These values should correspond with the forcing files you are using. Here the stop time is set to 72 hours or three days adn we start at tiem $t=0$. If we were running longer or restarting a simulation these values might change. 

**Dump Interval:** This key specifies the timing interval at which ParFlow time-dependent outputs will be written.  A positive value for dump interval will be interpreted as a time interval (in this case a real, or float in hours), a negative value will be intepreted as an *integer* value of the time step.  Here we a `DumpInteval` set to 1.0 *h* so we will have outputs written hourly.  If the solution process goes smoothly (without any failed timesteps that restart) this will write output for every hour of simulation.

**Time Step:** This is the $\Delta t$ used to advance our simulation in time. Here we use a constant time step of 1.0 *h*.

**Time Cycles:**  You can give certain periods of time names if you want. For example if you aren’t running with CLM and you would like to have periods with rain and periods without. Here, however we have only one time cycle because CLM will handle the variable forcings. Therefore, we specify one time cycle and it’s constant for the duration of the simulation. We accomplish this by giving it a repeat value of -1, which repeats indefinitely. The length of the cycle is the length specified below (an integer) multiplied by the base unit value we specified earlier.

In [16]:
model.TimingInfo.BaseUnit = 1.0
model.TimingInfo.StartCount = 0
model.TimingInfo.StartTime = 0
model.TimingInfo.StopTime = 72.0
model.TimingInfo.DumpInterval = 1.0
model.TimeStep.Type = "Constant"
model.TimeStep.Value = 1.0

#Time cycles
model.Cycle.Names ="constant"
model.Cycle.constant.Names = "alltime"
model.Cycle.constant.alltime.Length = 1
model.Cycle.constant.Repeat = -1

# An alternate approach defining rainfall and recession time periods are defined here
# rain for 1 hour, recession for 2 hours
# model.Cycle.Names ="constant rainrec"
# model.Cycle.rainrec.Names = "rain rec"
# model.Cycle.rainrec.rain.Length = 1
# model.Cycle.rainrec.rec.Length = 5000000
# model.Cycle.rainrec.Repeat = -1

## 7. Boundary and intial conditions

### 7.1 Boundary conditions
Now, we assign Boundary Conditions for each face (each of the Patches in the domain defined before). Recall the previously stated Patches and associate them with the boundary conditions that follow. The bottom and sides of our domain are all set to no-flow (i.e. constant flux of 0) boundaries. The top is set to an overland flow boundary to turn on the integrated overland flow routing, in this case, overland kinematic wave approximation.

In [17]:
model.BCPressure.PatchNames = "top bottom side"

model.Patch.bottom.BCPressure.Type = "FluxConst"
model.Patch.bottom.BCPressure.Cycle = "constant"
model.Patch.bottom.BCPressure.alltime.Value = 0.0

model.Patch.side.BCPressure.Type = "FluxConst"
model.Patch.side.BCPressure.Cycle = "constant"
model.Patch.side.BCPressure.alltime.Value = 0.0

model.Patch.top.BCPressure.Type = "OverlandKinematic"
model.Patch.top.BCPressure.Cycle = "constant"
model.Patch.top.BCPressure.alltime.Value = 0.0

### 7.2 Initial conditions: water pressure
Next we set the initial conditions for the domain. In this example we are using a pressure .pfb file that was obtained by spinning up the model in the workflow outlined in § 3.1.2.  Alternatively, the water table can be set to a constant value by changing the `ICPressure.Type` and setting a reference patch. Again, the input file that is referenced here was was copied into the run directory at the top of this script.

In [18]:
model.ICPressure.Type = "PFBFile"
model.ICPressure.GeomNames = "domain"
model.Geom.domain.ICPressure.RefPatch = "top"
model.Geom.domain.ICPressure.FileName = "press.init.pfb"

## 8. CLM Settings
In this example we are using ParFlow with the CLM land surface model so we must provide some parameters for CLM (§ 6.1.36). Note that CLM will also require some additional inputs file specific to the model (in this case `drv_climin.dat`, `drv_vegm.alluv.dat` and `drv_vegp.dat`)

**Outputs:** First we specify that we will be using CLM as the land surface model and provide the name of a directory that outputs will be written to.  For this example we do not need outputs for each processor or a binary output directory.  We set the dump interval to 1, indicating that we will be writing outputs for every time step. Note that this does not have to match the dump interval for ParFlow outputs. 

**Forcings:** Next we specify the details of the meteorological forcing files that clm will read.  First we provide the name of the files and the directory they can be found in. Next we specify that we are using 3D forcing files meaning that we have spatially distributed forcing with multiple time steps in every file. Therefore we must also specify the number of times steps (`MetFileNT`) in every file, in this case 24. Finally, we specify the initial value for the CLM counter.

**Pysical properties:** This last set of CLM parameters refers to the physical properties of the system. Refer to § 6.1.36 for details.

In [19]:
model.Solver.LSM = "CLM"

# outputs
model.Solver.CLM.CLMFileDir = "clm_output/"
model.Solver.CLM.Print1dOut = False
model.Solver.BinaryOutDir = False #Solver: Field BinaryOutDir is not part of the expected schema <class 'parflow.tools.database.generated.Solver'>
model.Solver.CLM.DailyRST = True
model.Solver.CLM.CLMDumpInterval = 1

# forcing files
model.Solver.CLM.MetFileName = "NLDAS"
model.Solver.CLM.MetFilePath = "../../NLDAS"
model.Solver.CLM.MetForcing = "3D"
model.Solver.CLM.MetFileNT = 24
model.Solver.CLM.IstepStart = 1

# physical properties
model.Solver.CLM.EvapBeta = "Linear"
model.Solver.CLM.VegWaterStress = "Saturation"
model.Solver.CLM.ResSat = 0.1
model.Solver.CLM.WiltingPoint = 0.12
model.Solver.CLM.FieldCapacity = 0.98
model.Solver.CLM.IrrigationType = "none"
model.Solver.CLM.RootZoneNZ = 4
model.Solver.CLM.SoiLayer = 4


Solver: Field BinaryOutDir is not part of the expected schema <class 'parflow.tools.database.generated.Solver'>


## 9. Solver options

### 9.1 Output control
Now we specify what outputs we would like written. In this example we specify that we would like to write out CLM variables as well as Pressure and Saturation. However, there are many options for this and you should change these options according to what type of analysis you will be performing on your results. A complete list of print options is provided in § 6.1.32.

In [20]:
model.Solver.PrintSubsurfData = True
model.Solver.PrintPressure = True
model.Solver.PrintSaturation = True
model.Solver.PrintMask = True
model.Solver.PrintVelocities = False
model.Solver.PrintEvapTrans = False
model.Solver.CLM.SingleFile = True
model.Solver.PrintSlopes = True

model.Solver.WriteCLMBinary = False
model.Solver.PrintCLM = True

### 9.2 Solver settings
Next we specify the solver settings for the ParFlow (§ 6.1.34). 

**Solver:** First we turn on solver Richards and the terrain following grid.

**Exact solution:** We don't have one here, the exact solution is used for error calculations in testing ParFlow against published analytical solutions.

**Solver settings:** We then set the max solver settings and linear and nonlinear convergence tolerance settings. The linear system will be solved to a norm of 1e−8 and the nonlinear system will be solved to a norm of less than 1e−6. Also of note is the EtaChoice of EtaConstant and that we use the analytical Jacobian (`UseJacobian=True`). We are also using the PFMG preconditioner.



In [21]:
# Solver types
model.Solver = "Richards"
model.Solver.TerrainFollowingGrid = True
model.Solver.TerrainFollowingGrid.SlopeUpwindFormulation = "Upwind" 
model.Solver.Linear.Preconditioner = "PFMG"
#model.Solver.Linear.Preconditioner.PCMatrixType = "FullJacobian"

# Exact solution
model.KnownSolution = "NoKnownSolution"

# Solver settings
model.Solver.MaxIter = 25000
model.Solver.Drop = 1e-20
model.Solver.AbsTol = 1e-8
model.Solver.MaxConvergenceFailures = 8
model.Solver.Nonlinear.MaxIter = 1000
model.Solver.Nonlinear.ResidualTol = 1e-6
model.Solver.Nonlinear.EtaChoice =  "EtaConstant"
model.Solver.Nonlinear.EtaValue = 0.001
model.Solver.Nonlinear.UseJacobian = True
model.Solver.Nonlinear.DerivativeEpsilon = 1e-16
model.Solver.Nonlinear.StepTol = 1e-15
model.Solver.Nonlinear.Globalization = "LineSearch"
model.Solver.Linear.KrylovDimension = 70
model.Solver.Linear.MaxRestarts = 2


## 10.  Distribute files, write the model and run the simulation

**Distribute:** Next we distribute all the inputs over our domain geometry. Note the slopes are 2D files, so we need to set our NZ to one and then back to 10 for the rest of the ParFlow inputs which are 3D. Note that this is an action that is taken and will result in the dist files being created. 

**Write:** The `model.write()` command *is optional* writes the ParFlow database that is used to run the model.  Here we are also writing `yaml` and `json` files. These files are not necessary to run the model but can be used as metadata with model simulations and to load simulations for analysis in Python. 

**Run:** Finally we actually run the model!  Note this is the step at which outputs will be generated in your outputs folder. 

In [22]:
# Distribute input files
# slope files are 2D (i.e. they only have one layer) so you need to set NZ to 1 before you distribute them
# Make sure to set it back to your actual NZ before distributing 3D files or running your model
#model.ComputationalGrid.NZ =1
model.dist("slopex_LW.pfb")
model.dist("slopey_LW.pfb")

#model.ComputationalGrid.NZ = 10 #the rest of the inputs shoul dbe distributed over 3D space
model.dist("Indicator_LW_USGS_Bedrock.pfb")
model.dist("press.init.pfb")

# write
model.write()
model.write(file_format='yaml')
model.write(file_format='json')

#run
model.run()


# ==============================================================================
# ParFlow directory
#  - /usr/local
# ParFlow version
#  - 3.10.0
# Working directory
#  - /data/outputs/LW_CLM
# ParFlow database
#  - LW_CLM.pfidb
# ==============================================================================


# ==============================================================================
# ParFlow ran successfully 💦 💦 💦 
# ==============================================================================

